In [ ]:
# Instalación de dependencias

%pip install -r requirements.txt

In [ ]:
# Experimento a ejecutar

experiment_name="EXPERIMENT_NAME"           # A completar
experiment_path="EXPERIMENT_PATH"           # A completar

In [ ]:
# Inicialización opcional de la sesión en Hugging Face (seguro)
import os

# Carga variables de entorno desde apis.env (si existe)
from dotenv import load_dotenv
load_dotenv("api.env")

# Leer token y realizar login sólo si el token está presente
api_key = os.getenv("HUGGINGFACE_TOKEN")
if api_key:
    try:
        from huggingface_hub import login
        login(token=api_key)
        print("Logged in to Hugging Face Hub")
    except Exception as e:
        print("Hugging Face login failed:", e)
else:
    print("No HUGGINGFACE_TOKEN found in apis.env — skipping Hugging Face login (public model access only)")

In [ ]:
# Cargar modelo

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "swiss-ai/Apertus-8B-Instruct-2509"
device = "cuda" if torch.cuda.is_available() else "cpu"  # for GPU usage or "cpu" for CPU usage

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
).to(device)

print(f"Modelo cargado en dispositivo: {device}")

In [ ]:
# Preparar experimento

from scripts.prepare_experiment import prepare_batches_for_experiment

prepare_batches_for_experiment(experiment_name, experiment_path)

In [ ]:
# Ejecutar experimento

from scripts.execute_experiment import run_experiment

# Ejecutar el experimento (procesa solo palabras pendientes)
df_results = run_experiment(
    experiment_name=experiment_name,
    experiment_path=experiment_path,
    model=model,
    tokenizer=tokenizer,
    device=device,
    do_sample=False 
)

print(f"\nTotal de palabras procesadas: {len(df_results)}")